# Carregando os dados da fonte

In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet(r"data/yellow_tripdata_2024.parquet")

In [4]:
print(f"Tamanho: {len(df)}")
df.head()

Tamanho: 123510


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:52:25,2024-01-01 01:06:20,1.0,2.30,1.0,N,79,229,1,14.9,1.0,0.5,2.98,0.0,1.0,22.88,2.5,0.0
1,2,2024-01-01 00:16:51,2024-01-01 00:30:01,2.0,1.85,1.0,N,107,229,2,12.8,1.0,0.5,0.00,0.0,1.0,17.80,2.5,0.0
2,2,2024-01-01 00:26:53,2024-01-01 00:39:50,1.0,1.85,1.0,N,144,164,1,13.5,1.0,0.5,3.70,0.0,1.0,22.20,2.5,0.0
3,2,2024-01-01 00:57:07,2024-01-01 01:03:54,2.0,0.69,1.0,N,161,237,1,7.9,1.0,0.5,1.00,0.0,1.0,13.90,2.5,0.0
4,2,2024-01-01 00:09:31,2024-01-01 00:34:30,2.0,5.73,1.0,N,144,236,1,28.2,1.0,0.5,6.64,0.0,1.0,39.84,2.5,0.0


# Particionando os dados

In [13]:
df['pickup_date'] = df['tpep_pickup_datetime'].dt.date

In [29]:
df_dia = {
    dia.strftime('%Y-%m-%d'): grupo for dia, grupo in df.groupby('pickup_date')
}

# Carregando na camada bronze

In [24]:
import boto3
from io import BytesIO

In [25]:
s3 = boto3.client('s3')

In [26]:
for key, value in df_dia.items():
    buffer = BytesIO()
    value.to_parquet(buffer, index=False, compression='snappy')
    buffer.seek(0)

    ano, mes, dia = key.split('-')
    s3.put_object(
    Bucket="uber-pipeline-2025",
    Key=f"bronze/{ano}/{mes}/{dia}/{key}.parquet",
    Body=buffer.getvalue()
)   

# Tratando os dados

In [5]:
df['trip_id'] = df.index

## Tabelas dimensão

### datetime_dim

In [6]:
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].copy()
datetime_dim['pick_minute'] = datetime_dim['tpep_pickup_datetime'].dt.minute
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['drop_minute'] = datetime_dim['tpep_dropoff_datetime'].dt.minute
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

datetime_dim['datetime_id'] = datetime_dim.index

datetime_dim = datetime_dim[[
    'datetime_id',
    'tpep_pickup_datetime', 'tpep_dropoff_datetime',
    'pick_minute', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
    'drop_minute', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday',
    ]]

In [7]:
datetime_dim

,datetime_id,tpep_pickup_datetime,tpep_dropoff_datetime,pick_minute,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_minute,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2024-01-01 00:52:25,2024-01-01 01:06:20,52,0,1,1,2024,0,6,1,1,1,2024,0
1,1,2024-01-01 00:16:51,2024-01-01 00:30:01,16,0,1,1,2024,0,30,0,1,1,2024,0
2,2,2024-01-01 00:26:53,2024-01-01 00:39:50,26,0,1,1,2024,0,39,0,1,1,2024,0
3,3,2024-01-01 00:57:07,2024-01-01 01:03:54,57,0,1,1,2024,0,3,1,1,1,2024,0
4,4,2024-01-01 00:09:31,2024-01-01 00:34:30,9,0,1,1,2024,0,34,0,1,1,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123505,123505,2024-12-31 23:15:58,2024-12-31 23:30:48,15,23,31,12,2024,1,30,23,31,12,2024,1
123506,123506,2024-12-31 23:28:13,2025-01-01 00:02:21,28,23,31,12,2024,1,2,0,1,1,2025,2
123507,123507,2024-12-31 23:07:11,2024-12-31 23:16:32,7,23,31,12,2024,1,16,23,31,12,2024,1
123508,123508,2024-12-31 23:38:10,2024-12-31 23:58:13,38,23,31,12,2024,1,58,23,31,12,2024,1


### trip_distance_dim

In [8]:
trip_distance_dim = df[['trip_distance']].drop_duplicates().reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']]

In [9]:
trip_distance_dim

,trip_distance_id,trip_distance
0,0,2.30
1,1,1.85
2,2,0.69
3,3,5.73
4,4,2.86
...,...,...
2608,2608,24.25
2609,2609,22.40
2610,2610,40.86
2611,2611,13.85


### passenger_count_dim

In [49]:
passenger_count_dim = df[['passenger_count']].fillna(1).drop_duplicates().reset_index(drop=True).astype('Int32')
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id', 'passenger_count']]

In [50]:
passenger_count_dim

,passenger_count_id,passenger_count
0,0,1
1,1,2
2,2,5
3,3,6
4,4,4
5,5,0
6,6,3
7,7,8


### rate_code_dim

In [12]:
df['RatecodeID'] = df['RatecodeID'].fillna(99).astype(int)

In [13]:
rate_code_type = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride",
    99: "Null/unknown"
}

rate_code_dim = df[['RatecodeID']].drop_duplicates().reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]

In [14]:
rate_code_dim

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,99,Null/unknown
2,2,5,Negotiated fare
3,3,2,JFK
4,4,3,Newark
5,5,4,Nassau or Westchester
6,6,6,Group ride


### payment_type_dim

In [15]:
payment_type_name = {
    0: "Flex Fare trip",
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}

payment_type_dim = df[['payment_type']].drop_duplicates().reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)

payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]

In [16]:
payment_type_dim

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,2,Cash
2,2,0,Flex Fare trip
3,3,4,Dispute
4,4,3,No charge


### vendor_dim

In [17]:
vendor_type_name = {
    1:"Creative Mobile Technologies, LLC",
    2:"Curb Mobility, LLC",
    6:"Myle Technologies Inc",
    7:"Helix"
}

vendor_dim = df[['VendorID']].drop_duplicates().reset_index(drop=True)
vendor_dim['vendor_id'] = vendor_dim.index
vendor_dim['vendor_type_name'] = vendor_dim['VendorID'].map(vendor_type_name)
vendor_dim = vendor_dim[['vendor_id', 'VendorID', 'vendor_type_name']]

In [18]:
vendor_dim

,vendor_id,VendorID,vendor_type_name
0,0,2,"Curb Mobility, LLC"
1,1,1,"Creative Mobile Technologies, LLC"
2,2,6,Myle Technologies Inc


## Tabela Fato

In [19]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,2,2024-01-01 00:52:25,2024-01-01 01:06:20,1.0,2.30,1,N,79,229,1,14.9,1.0,0.5,2.98,0.0,1.0,22.88,2.5,0.0,0
1,2,2024-01-01 00:16:51,2024-01-01 00:30:01,2.0,1.85,1,N,107,229,2,12.8,1.0,0.5,0.00,0.0,1.0,17.80,2.5,0.0,1
2,2,2024-01-01 00:26:53,2024-01-01 00:39:50,1.0,1.85,1,N,144,164,1,13.5,1.0,0.5,3.70,0.0,1.0,22.20,2.5,0.0,2
3,2,2024-01-01 00:57:07,2024-01-01 01:03:54,2.0,0.69,1,N,161,237,1,7.9,1.0,0.5,1.00,0.0,1.0,13.90,2.5,0.0,3
4,2,2024-01-01 00:09:31,2024-01-01 00:34:30,2.0,5.73,1,N,144,236,1,28.2,1.0,0.5,6.64,0.0,1.0,39.84,2.5,0.0,4


In [83]:
fact_table = df\
    .merge(vendor_dim, on='VendorID')\
    .merge(passenger_count_dim, on='passenger_count')\
    .merge(trip_distance_dim, on='trip_distance')\
    .merge(rate_code_dim, on='RatecodeID')\
    .merge(datetime_dim, on=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])\
    .merge(payment_type_dim, on='payment_type')\
    [['trip_id', 'vendor_id', 'datetime_id', 'passenger_count_id', 'trip_distance_id',
      'rate_code_id', 'payment_type_id', 'fare_amount', 'extra',
      'mta_tax', 'tip_amount', 'tolls_amount',
      'improvement_surcharge', 'total_amount']]
                

In [84]:
fact_table

,trip_id,vendor_id,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,0,0,0,0,0,0,14.9,1.0,0.5,2.98,0.00,1.0,22.88
1,1,0,1,1,1,0,1,12.8,1.0,0.5,0.00,0.00,1.0,17.80
2,2,0,2,0,1,0,0,13.5,1.0,0.5,3.70,0.00,1.0,22.20
3,3,0,3,1,2,0,0,7.9,1.0,0.5,1.00,0.00,1.0,13.90
4,4,0,4,1,3,0,0,28.2,1.0,0.5,6.64,0.00,1.0,39.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111429,123504,0,123504,6,1306,0,1,32.4,1.0,0.5,0.00,0.00,1.0,37.40
111430,123505,0,123505,0,578,0,0,15.6,1.0,0.5,2.00,0.00,1.0,22.60
111431,123506,0,123506,0,2207,0,1,81.4,1.0,0.5,0.00,6.94,1.0,93.34
111432,123507,0,123507,4,377,0,0,10.7,1.0,0.5,1.57,0.00,1.0,17.27


## Carregando as tabelas na camada silver

In [23]:
import boto3
from io import BytesIO

In [85]:
tables = {
    'datetime_dim': datetime_dim,
    'trip_distance_dim': trip_distance_dim,
    'passenger_count_dim': passenger_count_dim,
    'rate_code_dim': rate_code_dim,
    'payment_type_dim': payment_type_dim,
    'vendor_dim': vendor_dim,
    'fact_table': fact_table
}

In [24]:
s3 = boto3.client('s3')

In [26]:
for key, value in tables.items():
    buffer = BytesIO()
    value.to_parquet(buffer, index=False, compression='snappy')
    buffer.seek(0)

    s3.put_object(
    Bucket="uber-pipeline-2025",
    Key=f"silver/{key}/{key}.parquet",
    Body=buffer.getvalue()
)   